# Analyze Item-Item Results

This notebook analyzes the item-item CF results.

## Setup

In [ ]:
library(dplyr)
library(ggplot2)
library(reshape2)
options(repr.plot.height=5)

In [ ]:
results = read.csv("build/eval-results.csv")
head(results)

## Utility Functions

Compute the standard error:

In [ ]:
se = function(x) {
    sd(x) / sqrt(length(x))
}

Plot a metric, with its error bars:

In [ ]:
plot.metric = function(data) {
  ggplot(data) +
    aes(x=NNbrs, color=Algorithm, shape=Algorithm, y=Value) +
    geom_line() + geom_point() +
    geom_errorbar(aes(ymax=Value + Err, ymin=Value - Err)) +
    facet_wrap(~ DataSet)
}

## Results

In [ ]:
results.grouped = results %>% group_by(DataSet, Algorithm, NNbrs)

In [ ]:
results.grouped %>%
    summarize(Value=mean(Coverage), Err=se(Coverage)) %>%
    plot.metric()

In [ ]:
rmse.data = results %>%
    select(Algorithm, Partition, NNbrs, RMSE.ByRating, RMSE.ByUser) %>%
    melt(id.vars=c("Algorithm", "Partition", "NNbrs"), variable.name="Metric") %>%
    group_by(Algorithm, NNbrs, Metric) %>%
    summarize(mean=mean(value))
ggplot(rmse.data %>% filter(!is.na(NNbrs))) +
    aes(x=NNbrs, color=Algorithm, shape=Algorithm, y=mean) +
    geom_line() + geom_point() +
    geom_hline(data=(rmse.data %>% filter(is.na(NNbrs))),
                mapping=aes(yintercept=mean)) +
    facet_wrap(~ Metric)

In [ ]:
results.grouped %>%
    summarize(Value=mean(RMSE.ByUser), Err=se(RMSE.ByUser)) %>%
    plot.metric()

In [ ]:
results.grouped %>%
    summarize(Value=mean(TopN.nDCG), Err=se(TopN.nDCG)) %>%
    plot.metric()

In [ ]:
results.grouped %>%
    summarize(Value=mean(Predict.nDCG), Err=se(Predict.nDCG)) %>%
    plot.metric()